TEXT CLEANING

In [12]:
import re
import string
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MB511WS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
def clean_text(text):
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # convert to lowercase
    text = text.lower()

    # remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])

    # remove special character
    text = re.sub(r'\W+', ' ', text)

    return text


sample_text = "Client been very busy - unable to discuss Also still meeting with clients about onboarding them Awaiting further feedback"
cleaned_text = clean_text(sample_text)
print(cleaned_text)


client busy unable discuss also still meeting clients onboarding awaiting feedback


TOKENIZATION

In [14]:
from nltk.tokenize import sent_tokenize, word_tokenize

def tokenize_word(text) :
    word_token = word_tokenize(text)
    sent_token = sent_tokenize(text)

    return word_token, sent_token

word_token, sent_token = tokenize_word(cleaned_text)
print("word token", word_token)
print("sentence token", sent_token)

word token ['client', 'busy', 'unable', 'discuss', 'also', 'still', 'meeting', 'clients', 'onboarding', 'awaiting', 'feedback']
sentence token ['client busy unable discuss also still meeting clients onboarding awaiting feedback']


LEMMATIZATION

In [15]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmentizer = WordNetLemmatizer()

def lemmantize_text(text):
    tokens = word_tokenize(text)
    lemmantize_token = [lemmentizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmantize_token)

lemmatized_text = lemmantize_text(cleaned_text)
print("Lemmatized Text:", lemmatized_text)

Lemmatized Text: client busy unable discus also still meeting client onboarding awaiting feedback


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MB511WS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\MB511WS\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


TEXT REPRESENTATION

In [16]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Bag of Words
vectorizer = CountVectorizer()
X_bow = vectorizer.fit_transform([lemmatized_text])
print("Bag of Words representation:\n", X_bow.toarray())

# TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform([lemmatized_text])
print("TF-IDF representation:\n", X_tfidf.toarray())


Bag of Words representation:
 [[1 1 1 2 1 1 1 1 1 1]]
TF-IDF representation:
 [[0.2773501 0.2773501 0.2773501 0.5547002 0.2773501 0.2773501 0.2773501
  0.2773501 0.2773501 0.2773501]]


FEATURE ENGINEERING

In [22]:
vectorizer_ngram = CountVectorizer(ngram_range=(1, 2))
X_ngram = vectorizer_ngram.fit_transform([lemmatized_text])
print("N-gram representation:\n", X_ngram.toarray())

N-gram representation:
 [[1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1]]


In [23]:
import spacy
nlp = spacy.load('en_core_web_sm')

def pos_tagging(text):
    doc = nlp(text)
    return [(token.text, token.pos_) for token in doc]

pos_tags =  pos_tagging(lemmatized_text)
print("POS Tags", pos_tags)

POS Tags [('client', 'NOUN'), ('busy', 'ADJ'), ('unable', 'ADJ'), ('discus', 'NOUN'), ('also', 'ADV'), ('still', 'ADV'), ('meeting', 'VERB'), ('client', 'NOUN'), ('onboarding', 'NOUN'), ('awaiting', 'VERB'), ('feedback', 'NOUN')]


In [24]:
def name_entity_recognition(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

entities = name_entity_recognition(lemmatized_text)
print("Named Entities", entities)

Named Entities []


HANDLING IMBALANCE AND NOISE

In [25]:
import nlpaug.augmenter.word as naw

def augment_text(text):
    aug = naw.SynonymAug(aug_src="wordnet")
    augmented_text = aug.augment(text)
    return augmented_text

augmented_text = augment_text(lemmatized_text)
print("Augemented Text: ", augmented_text)

Augemented Text:  ['client busy ineffectual discus besides still meet node onboarding awaiting feedback']


DEEP LEARNING FOR NLP

In [27]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

# load pretrained model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# text classification
nlp_classify = pipeline('text-classification', model = model, tokenizer = tokenizer)
result = nlp_classify(augmented_text)

print(result)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'label': 'LABEL_1', 'score': 0.5187152028083801}]


In [1]:
from gensim import corpora, models, similarities

# Example corpus
documents = ["This is a fantastic product!", "I love this product.", "I do not like this product.", "This is a bad product."]

# Tokenize and create dictionary
texts = [[word for word in document.lower().split()] for document in documents]
dictionary = corpora.Dictionary(texts)

# Create the bag-of-words corpus
corpus = [dictionary.doc2bow(text) for text in texts]

# Train LDA model
lda = models.LdaModel(corpus, num_topics=2, id2word=dictionary)

# Print topics
for idx, topic in lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

# Categorize new document
new_doc = "I love this fantastic product"
new_bow = dictionary.doc2bow(new_doc.lower().split())
print(lda.get_document_topics(new_bow))


Topic: 0 
Words: 0.171*"this" + 0.120*"product." + 0.104*"i" + 0.089*"is" + 0.078*"a" + 0.069*"fantastic" + 0.068*"product!" + 0.066*"do" + 0.065*"love" + 0.063*"not"
Topic: 1 
Words: 0.134*"product." + 0.130*"this" + 0.121*"a" + 0.102*"is" + 0.093*"bad" + 0.076*"i" + 0.064*"like" + 0.062*"not" + 0.058*"love" + 0.057*"do"
[(0, 0.876012), (1, 0.12398801)]


In [3]:
from transformers import pipeline

# Load the text classification pipeline
classifier = pipeline("text-classification", model="bert-base-uncased", tokenizer="bert-base-uncased")

# Define your text for classification
text = "currently 5 clients using qbo"

# Perform text classification
result = classifier(text)

# Print the result
print("Predicted label:", result[0]['label'])

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


c:\Users\MB511WS\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\MB511WS\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.3.0+cu121 with CU

Predicted label: LABEL_0
